In [19]:
import sys
import pandas as pd

import transformers
from transformers import BertTokenizer

import torch

from torch.utils.data import DataLoader

In [37]:
f = "../../finphrase_dir/Sentences_75Agree.txt"
dataset = pd.read_csv(f, sep='\@', engine='python', header=None, names=['sentence', 'label'])
# dataset.drop_duplicates(inplace=True)
dataset.reset_index(inplace=True)

train = dataset['sentence']
label = dataset['label']

train_tensor = torch.utils.data.TensorDataset(train, label)

data_loader = DataLoader(train_tensor, batch_size=1, shuffle=True)
for x, batch in enumerate(data_loader):
    print(batch)
# print(data_loader)

TypeError: 'int' object is not callable

In [3]:

# Load the tokenizer of the "bert-base-cased" pretrained model
# See https://huggingface.co/transformers/pretrained_models.html for other models
tz = BertTokenizer.from_pretrained("bert-base-cased")

# The senetence to be encoded
sent = "Let's learn deep learning!"

# Encode the sentence
encoded = tz.encode_plus(
    text=sent,  # the sentence to be encoded
    add_special_tokens=True,  # Add [CLS] and [SEP]
    max_length = 64,  # maximum length of a sentence
    pad_to_max_length=True,  # Add [PAD]s
    return_attention_mask = True,  # Generate the attention mask
    return_tensors = 'pt',  # ask the function to return PyTorch tensors
)

# Get the input IDs and attention mask in tensor format
input_ids = encoded['input_ids']
attn_mask = encoded['attention_mask']

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [4]:
input_ids

tensor([[ 101, 2421,  112,  188, 3858, 1996, 3776,  106,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])

In [5]:
attn_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [6]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

In [10]:
class BertToSentiment(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            BertModel.from_pretrained("bert-base-cased", output_hidden_states=True, output_attentions=True),
            nn.ReLU(),
#             print("1"),
            nn.Linear(32, 128),
            nn.ReLU(),
#             print("2"),
            nn.Linear(128, 128),
            nn.ReLU(),
#             print("3"),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.25),
#             print("4"),
            nn.Linear(64, 3), 
#             print("5"),
        )
#         self.input_layer = nn.Linear(32, hidden_size) #INput size is probabl wrong, adjust to output of BERT
#         self.hidden1 = nn.Linear(hidden_size, hidden_size)
#         self.hidden2 = nn.Linear(hidden_size, hidden_size)
#         self.softmax = nn.Softmax(dim=None)
#         self.output = nn.Linear(hidden_size, output_size)
#         self.bert = BertModel.from_pretrained("bert-base-cased", output_hidden_states=True, output_attentions=True)
        
    def forward(self, x):
#         x = F.relu(self.bert(x))
#         x = F.relu(self.input_layer(x))
#         x = F.relu(self.hidden1(x))
#         x = F.relu(self.hidden2(x))
#         x = self.softmax(self.output(x))
#         return x
        return self.model(x)
    

In [11]:
#model = BertToSentiment(10, 3)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
inputs = tokenizer("GME is a very very bad stock. Sell this immediately", 
                   return_tensors="pt", 
                   pad_to_max_length=True, 
                   max_length = 64,
                   add_special_tokens=True,
                   padding_side='right')
inputs['input_ids'].shape


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Keyword arguments {'padding_side': 'right'} not recognized.


torch.Size([1, 64])

In [12]:
model = BertToSentiment()

In [30]:
#bert_model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)


def train_model(model, data):
    num_epochs=2
    batch_size = 1
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    model = model.to(device)
    model.train()
    
    training_acc = []
    validation_acc = []
    
    for epoch in range(num_epochs):  # loop over the dataset multiple times
#         print(epoch)
        training_losses = []
        
        for vector in data:
            x, y = vector, [0,1,0]
            print(x.size(), y)
        # get the inputs; data is a list of [inputs, labels]
            x, y = x.to(device), x.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(x)
            loss = criterion(outputs, x)
            loss.backward()
            optimizer.step()

            
#         num_correct = 0
#         for i, data in enumerate(dataloaders['val']):
#             inputs, labels = data
#             inputs, labels = inputs.to(device), labels.to(device)

#             output = model(inputs)
#             _, preds = torch.max(output, 1)
#             num_correct += torch.sum(preds == labels)
#         val_acc_history.append(num_correct / (32 * len(dataloaders['val'])))

    print('Finished Training')
    
    ### -- Get model predictions, compute loss, call optimizer
    ### -- Get model predictions on validation data, compute accuracy, add to `val_acc_history`
    
    ### End TODO
    return model, val_acc_history

In [31]:
train_model(model, 2, inputs['input_ids'])

0
torch.Size([32]) [0, 1, 0]


ValueError: not enough values to unpack (expected 2, got 1)

In [24]:
print(inputs)

{'input_ids': tensor([[  101, 14748,  2036,  1110,   170,  1304,  1304,  2213,  4482,   119,
         22087,  2339,  1142,  2411,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])}


In [25]:
print(input_ids.size())

torch.Size([1, 64])
